# 🎤 RVC Voice Cloning System - Google Colab

This notebook provides GPU access for users without local GPUs.

**All logic comes from the GitHub repository** - this notebook only orchestrates execution.

## Setup Instructions

1. Run all cells in order
2. The notebook will clone the repo and install dependencies
3. Use the training and inference cells below
4. Optionally launch Gradio UI for interactive use

## 📦 Step 1: Clone Repository and Install Dependencies

In [ ]:
# Clone the repository
import os
import subprocess

# Replace with your actual GitHub repository URL
REPO_URL = "https://github.com/yourusername/rvc-system.git"
REPO_DIR = "rvc-system"

# Clone if not already cloned
if not os.path.exists(REPO_DIR):
    print(f"Cloning repository from {REPO_URL}...")
    subprocess.run(["git", "clone", REPO_URL, REPO_DIR], check=True)
    print("✅ Repository cloned successfully")
else:
    print(f"Repository already exists at {REPO_DIR}")

# Change to repository directory
os.chdir(REPO_DIR)
print(f"Working directory: {os.getcwd()}")

In [ ]:
# Install dependencies
print("Installing dependencies...")
subprocess.run(["pip", "install", "-r", "requirements.txt"], check=True)
print("✅ Dependencies installed")

In [ ]:
# Check GPU availability
import torch

if torch.cuda.is_available():
    print(f"✅ CUDA GPU available: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    device = torch.device("cuda")
else:
    print("⚠️  No GPU detected. Training will be slow on CPU.")
    device = torch.device("cpu")

print(f"Using device: {device}")

## 📥 Step 2: Download Pretrained Models (First Time Only)

In [ ]:
# Download pretrained models
print("Downloading pretrained models...")
subprocess.run(["python", "scripts/download_pretrained.py"], check=True)
print("✅ Pretrained models downloaded")

## 🎓 Step 3: Train a New Voice

Upload audio files and train a custom voice model.

In [ ]:
# Import core training function
from core.training import train_rvc_model, TrainingConfig
from utils.device import get_device
from utils.registry import VoiceRegistry

# Setup
device = get_device()
registry = VoiceRegistry(models_dir="models")

# Training parameters
PERSON_NAME = "my_voice"  # Change this to your desired voice name
AUDIO_FILES = [
    # Add paths to your uploaded audio files here
    # Example: "/content/audio1.wav", "/content/audio2.wav"
]

# Training config
config = TrainingConfig(
    epochs=50,
    batch_size=4,
    learning_rate=0.0001,
)

print(f"Starting training for voice: {PERSON_NAME}")
print(f"Audio files: {AUDIO_FILES}")
print(f"Device: {device}")

# Train model
model_path = train_rvc_model(
    audio_files=AUDIO_FILES,
    person_name=PERSON_NAME,
    output_dir="models",
    config=config,
    device=device,
)

# Register voice
registry.register_voice(
    person_name=PERSON_NAME,
    model_path=model_path,
)

print(f"✅ Training completed! Model saved to: {model_path}")

## 🎭 Step 4: Voice Conversion

Convert audio to a trained voice.

In [ ]:
# Import inference function
from core.inference import VoiceConverter
from utils.registry import discover_voices

# List available voices
available_voices = discover_voices(models_dir="models")
print(f"Available voices: {available_voices}")

# Conversion parameters
SOURCE_AUDIO = "/content/source_audio.wav"  # Path to source audio
TARGET_VOICE = available_voices[0] if available_voices else None  # Select voice
OUTPUT_PATH = "/content/output_converted.wav"

if TARGET_VOICE:
    print(f"Converting audio to voice: {TARGET_VOICE}")
    
    # Load converter
    model_path = registry.get_model_path(TARGET_VOICE)
    converter = VoiceConverter(model_path, device=device)
    
    # Convert
    converter.convert(
        source_audio_path=SOURCE_AUDIO,
        output_path=OUTPUT_PATH,
        pitch_shift=0.0  # Optional pitch shift in semitones
    )
    
    print(f"✅ Conversion completed! Output saved to: {OUTPUT_PATH}")
else:
    print("❌ No trained voices available. Please train a voice first.")

## 🖥️ Step 5: Launch Gradio UI (Optional)

Launch the Gradio UI for interactive use. This will create a public link.

In [ ]:
# Launch Gradio UI
import subprocess
import sys

print("Launching Gradio UI...")
print("This will create a public link that you can share.")
print("Press Ctrl+C to stop the server.")

# Modify app.py to enable share mode, or run with environment variable
os.environ["GRADIO_SHARE"] = "True"
subprocess.run([sys.executable, "app.py"])

## 📚 Additional Notes

- All ML logic lives in the `core/` directory
- This notebook only orchestrates execution
- Trained models are saved in `models/` directory
- You can download trained models for use elsewhere
- The same code runs identically in local GPU and Colab modes